# Sentiment Analysis

Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import spacy
import collections
import operator
from tqdm.autonotebook import tqdm

### Dataset

Loading the [Sentiment140](https://www.kaggle.com/kazanova/sentiment140) Dataset that includes 1.6 milion tweets

In [ ]:
%%time
columns = ["target", "ids", "date", "flag", "user", "text"]
encoding = "ISO-8859-1"
dataset = pd.read_csv('dataset/tweets.csv', encoding=encoding, names=columns)

Dropping columns with not useful data

In [ ]:
%%time
dataset.drop(columns=["ids", "date", "flag", "user"], inplace=True)
dataset.head()

Since the original dataset is using classes `0-Negative` and `4-Positive`, I am mapping the `4` to `1` to fix the confusing naming

In [ ]:
%%time
dataset.target.replace({4: 1}, inplace=True)

Now lets shuffle the rows in dataset, since originally they are sorted by the class

In [ ]:
dataset = dataset.sample(frac=1).reset_index(drop=True)
# Remove for final training
dataset = dataset.iloc[:50000]

In [ ]:
dataset.head()

### Preprocessing

Dataset is now ready for preprocessing. I will use [spacy](https://spacy.io) which is a very popular library for Natural Language Processing.

Firstly, lets load a model that will help us with tokenisation and lemmatisation of tweets included in the dataset

In [ ]:
nlp = spacy.load("en_core_web_lg")

Piping through the tweets from dataset, will result in a iterator object that we will need to use to iterate over preprocessed tweets - and tokenized words in each tweet.

In [ ]:
tweets_iterator = nlp.pipe(dataset.text, n_threads=-1, batch_size=32)

Lets find out which are the most popular words in the dataset by mapping them to a dictionary. This can be useful during the optimalisation process, if we would want to drop some rare words to not include them in the final model to reduce its size. This process can take quite a while, since we are operating on a very large dataset.

In [ ]:
words = collections.defaultdict(int)
for tweet in tqdm(tweets_iterator, total=dataset.shape[0]):
    for token in tweet:
        if not token.is_stop:
            lexeme = nlp.vocab[token.lemma]
            if lexeme.has_vector:
                words[lexeme] += 1

Sorting the counts, with its corresponding tokens

In [ ]:
sorted_words = sorted(words.items(), key=operator.itemgetter(1), reverse=True)

Now, once we have the tokens, sorted by their popularity, we will need to create the embedding matrix. The matrix consists of vectors for each token/word that we get from spacy. One important thing to notice is that we will make an extra entry in the matrix at index 0, that will represent the placeholder for empty word. To train the model (and to recieve the prediction) we will feed it with array of ids of words from the tweet. These array will need to be equal lenght, because neural network will always expect the input to be equal size. We will use this row `0` to refer to a empty token, so we can pad the input array to match the expected by neural network shape of the input.

In [ ]:
embedding = np.zeros((len(sorted_words) + 1, 300))
for (i, lexeme) in enumerate(sorted_words, start=1):
    embedding[i] = lexeme[0].vector

The resulting matrix is a very important part of the model, since it will serve as a weights matrix in the first layer of the model.

In [ ]:
embedding

### Model

At this point, once we are ready with the preprocessing data, lets design the architecture for our model. I will use [Keras](https://keras.io), which is a very popular Deep Learning library.

The model is a very simple Convolutional Neural Network consisting of 9 layers.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(embedding), 300, weights=[embedding], input_length=100),
    tf.keras.layers.Conv1D(16, kernel_size=3),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(32),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1)
])
model.summary()